In [18]:
import pandas as pd
import re
from unidecode import unidecode
import json
from typing import Dict,List
import os 

In [19]:
data_raw=[]
test_data_raw=[]
pattern=re.compile(r"https://www\.rottentomatoes\.com/m/([^/]+)")
grade_exception=set()
grade_map = {
    "F-": 0,
    "F": 1,
    "F+": 2,
    "D-": 3,
    "D": 4,
    "D+": 5,
    "C-": 6,
    "C": 7,
    "C+": 8,
    "B-": 9,
    "B": 10,
    "B+": 11,
    "A-": 12,
    "A": 13,
    "A+": 14
}
def parseGrade(grade:str):
    if grade is None:
        return None
    grade=grade.replace(" ","")
    try:
        grade=float(grade)
        max_grade=1
        while(max_grade<grade):
            max_grade*=10
        return grade/max_grade
    except:
        pass
    num_score=re.search(r"(?P<first>(\d+\.*\d*)|(\.\d+))/(?P<second>(\d+\.*\d*)|(\.\d+))",grade)
    if num_score and num_score.group("first") and num_score.group("second"):
        try:
            return float(num_score.group("first"))/float(num_score.group("second"))
        except:
            return grade
    alphabet_score=re.search(r"(\w) *(\+|-)*",grade)
    if alphabet_score and alphabet_score.group(1):
        try:
            submark=alphabet_score.group(2)
        except:
            pass
        if not submark:
            submark=""
        _abcgrade=alphabet_score.group(1).upper()+submark
        if _abcgrade in grade_map:
            return grade_map[_abcgrade]/grade_map["A+"]
    grade_exception.add(grade)
    # print(f"\{grade}\\")
    return grade
with open("review_list","r") as f:
    for line in f:
        movie:Dict=json.loads(line)
        for k,v in movie.items():
            movie=pattern.search(k)
            if movie is not None:
                movie=movie[1]
            _raw=[]
            for review in v:
                additional:List[str]=[r.strip() for r in review["additional"].split("|")]
                grade,date=None,None
                if len(additional)==3:
                    try:
                        _grade=re.search(r"Original Score: +(.+)",additional[1]).group(1)
                        grade=parseGrade(_grade)
                        if(grade and type(grade)!= str and grade>1 ):
                            # print("Discard parsed",_grade)
                            grade=_grade
                        date=additional[2]
                    except:
                        date=additional[2]
                        print(date)
                elif len(additional)==2:
                    date=additional[1]
                else:
                    raise NotImplemented
                review.update({"date":date,"grade":grade})
                _raw.append(dict(review,**{"movie":movie}))
            if len(_raw)>1:
                test_data_raw.append(_raw[-1])
                _raw=_raw[:-1]
            data_raw.extend(_raw)

Jun 5, 2023


In [20]:
df=pd.DataFrame(data_raw)
df

,critic,publication,state,review,additional,date,grade,movie
0,Frederic and Mary Ann Brussat,Spirituality & Practice,fresh,A sense-luscious triumph with its exquisite da...,"Full Review | Original Score: 4/5 | Jul 25, 2005","Jul 25, 2005",0.8,riverdance_the_show
1,Mike Massie,Gone With The Twins,rotten,At least it provides a constant source of unin...,"Full Review | Original Score: 2/10 | Nov 26, 2020","Nov 26, 2020",0.2,lake_placid_2
2,Tim Brayton,Alternate Ending,rotten,"As formulaic killer animal films go, any halfw...","Full Review | Original Score: 2/5 | Jun 5, 2020","Jun 5, 2020",0.4,lake_placid_2
3,Ed Travis,Hollywood Jesus,rotten,"Do yourselves a favor, skip Lake Placid 2.","Full Review | Apr 10, 2020","Apr 10, 2020",None,lake_placid_2
4,Brian Orndorf,Blu-ray.com,rotten,Gone are the cheeky impulses and amusing thesp...,"Full Review | Original Score: D | Nov 12, 2017","Nov 12, 2017",0.285714,lake_placid_2
...,...,...,...,...,...,...,...,...
1119721,Chuck Rudolph,Matinee Magazine,fresh,What promises to be a simple rehash of romanti...,"Full Review | Original Score: B- | Mar 6, 2001","Mar 6, 2001",0.642857,when_brendan_met_trudy
1119722,Frank Swietek,One Guy's Opinion,rotten,"Cheerful enough, but it really doesn't tell us...","Full Review | Original Score: C+ | Mar 5, 2001","Mar 5, 2001",0.571429,when_brendan_met_trudy
1119723,Christopher Null,Filmcritic.com,fresh,One of the liveliest comedies to come along in...,"Full Review | Original Score: 4/5 | Feb 20, 2001","Feb 20, 2001",0.8,when_brendan_met_trudy
1119724,Emanuel Levy,Variety,rotten,Recreations of scenes from Breathless and The ...,"Full Review | Feb 7, 2001","Feb 7, 2001",None,when_brendan_met_trudy


In [21]:
test_df=pd.DataFrame(test_data_raw)
test_df

,critic,publication,state,review,additional,date,grade,movie
0,Scott Weinberg,FEARnet,rotten,"Follow the basic ""killer croc"" template if you...","Full Review | Original Score: 2/5 | Jan 29, 2008","Jan 29, 2008",0.4,lake_placid_2
1,Dennis Harvey,Variety,fresh,Cluttered and downbeat but illuminating...,"Full Review | Apr 22, 2020","Apr 22, 2020",None,planet_of_the_humans
2,Elena de la Torre,Cine-Mundial,fresh,"Cecil B. DeMille has brought to the screen, in...","Full Review | Sep 18, 2019","Sep 18, 2019",None,the_story_of_dr_wassell
3,James Berardinelli,ReelViews,fresh,,"Full Review | Original Score: 3/4 | Jan 1, 2000","Jan 1, 2000",0.75,burnt_by_the_sun
4,Reza Noorani,The Times of India,rotten,"Even if you are weak-hearted, this one will no...","Full Review | Original Score: 2/5 | Jan 13, 2018","Jan 13, 2018",0.4,1921
...,...,...,...,...,...,...,...,...
39797,Alan Jones,Radio Times,rotten,"A dourly humourless, futuristic farrago.","Full Review | Original Score: 2/5 | May 5, 2011","May 5, 2011",0.4,10009274-priest
39798,Emma Dibdin,Digital Spy,rotten,In trying desperately (and understandably) to ...,"Full Review | Original Score: 2/5 | Sep 5, 2013","Sep 5, 2013",0.4,diana_2013
39799,Peter Debruge,Variety,rotten,Part one of a trilogy that may never see compl...,"Full Review | Apr 8, 2011","Apr 8, 2011",None,atlas_shrugged_part_i
39800,Boston Herald Staff,Boston Herald,fresh,,"Full Review | Original Score: 3/4 | Jan 1, 2000","Jan 1, 2000",0.75,wattstax


In [22]:
pattern=re.compile(r"https://www\.rottentomatoes\.com/m/([^/]+)")
metadatas=[]
with open("movie_list_enriched","r") as f:
    for line in f:
        meta:Dict=json.loads(line)
        for k,v in meta.items():
            title=pattern.findall(k)
            if not title:
                continue
            title=title[0]
            submeta=v.pop("meta",None)
            temp=dict(v,**{"movie":title})
            if submeta:
                for item in submeta:
                    descriptions=re.findall(r"[^:]+",item)
                    if len(descriptions)>=2:
                        temp.update({descriptions[0]:' '.join(descriptions[1:])})
                    else:
                        print(item)
            metadatas.append(temp)


In [23]:
metadf=pd.DataFrame(metadatas)
metadf=metadf[["movie","description","Genre","Original Language","Director","Runtime"]]
metadf=metadf.drop_duplicates(["movie"])
metadf

,movie,description,Genre,Original Language,Director,Runtime
0,riverdance_the_show,,NaN,NaN,John McColgan,NaN
1,lake_placid_2,"A sheriff, a big-game hunter and a wildlife of...","Horror, Mystery & thriller",English,David Flores,1h 36m
2,planet_of_the_humans,"Filmmaker Jeff Gibbs examines climate change, ...",Documentary,English,Jeff Gibbs,1h 40m
3,in-love-and-war2001,British soldier Eric Newby (Callum Blue) is ca...,"Romance, War",English,John Kent Harrison,1h 30m
4,the_story_of_dr_wassell,This film presents a dramatization of the wart...,Biography,English,Cecil B. DeMille,2h 20m
...,...,...,...,...,...,...
55413,encounter_point,Filmmakers Ronit Avni and Julia Bacha examine ...,Documentary,Arabic,"Ronit Avni, Julia Bacha",1h 28m
55414,braddock_missing_in_action_3,While Col. James Braddock (Chuck Norris) was s...,"Action, War",English,Aaron Norris,1h 43m
55415,50_first_kisses,,NaN,NaN,Yûichi Fukuda,NaN
55416,tom_of_your_life,"When a baby is born who inexplicably grows, ev...","Comedy, Drama, Sci-fi",English,Jeremy Sklar,1h 32m


In [29]:
arr=[0,1,2,4,5]
print(arr[:1]+arr[1+1:])

[0, 2, 4, 5]


In [24]:
# metadfview=metadf.__deepcopy__()
# metadfview=metadfview[metadfview["Original Language"].str.contains("English",na=False)]
# merged_df=df.merge(metadf,how="inner",on="movie")
# merged_df.to_csv("data.csv")

In [25]:
metadfview=metadf.__deepcopy__()
dfview=df.merge(metadfview[["Original Language","movie"]],how="inner",on="movie").drop(["Original Language"],axis=1)
os.makedirs("filtered",exist_ok=True)
dfview.to_csv("./filtered/review_list")
metadfview.to_csv("./filtered/metadata")

In [26]:
metadfview=metadf.__deepcopy__()
dfview=test_df.merge(metadfview[["Original Language","movie"]],how="inner",on="movie").drop(["Original Language"],axis=1)
os.makedirs("filtered_test",exist_ok=True)
dfview.to_csv("./filtered_test/review_list")
metadfview.to_csv("./filtered_test/metadata")